In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI RAG Engine with Weaviate

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/rag-engine/rag_engine_weaviate.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Frag-engine%2Frag_engine_weaviate.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/rag-engine/rag_engine_weaviate.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/rag-engine/rag_engine_weaviate.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/rag-engine/rag_engine_weaviate.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/rag-engine/rag_engine_weaviate.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/rag-engine/rag_engine_weaviate.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/rag-engine/rag_engine_weaviate.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/rag-engine/rag_engine_weaviate.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            

| | |
|-|-|
| Author(s) | [Ming Zhang](https://github.com/mzhang-ai) |

## Overview

This notebook illustrates how to use [Vertex AI RAG Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/rag-overview) with [Weaviate](https://weaviate.io/) as a vector database.

For more information, refer to the [official documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/use-weaviate-db).

For more details on RAG corpus/file management and detailed support please visit [Vertex AI RAG Engine API](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/rag-api)

## Get started

### Install Vertex AI SDK and other required packages


In [30]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [31]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [3]:
# Use the environment variable if the user doesn't provide Project ID.
import os

import vertexai

PROJECT_ID = "ragdemo-453517"  # @param {type:"string", isTemplate: true}
if PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Create a RAG corpus using Weaviate as the Vector Database

### Import libraries

In [4]:
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool

### Load embedding model and create RAG Config

In [5]:
# Configure a Google first-party embedding model
embedding_model_config = rag.EmbeddingModelConfig(
    publisher_model="publishers/google/models/text-embedding-004"
)

# Name your corpus
DISPLAY_NAME = "IBS"  # @param {type:"string", "placeholder": "your-corpus-name"}

# Configure a Weaviate Vector Database Instance for the corpus
# More details for how to deploy a Weaviate Database Instance
# https://cloud.google.com/vertex-ai/generative-ai/docs/use-weaviate-db
WEAVIATE_HTTP_ENDPOINT = (
    "https://qdydcws8aopshqoyausw.c0.europe-west3.gcp.weaviate.cloud"  # @param {type:"string", "placeholder": "your-weaviate-http-endpoint"}
)
COLLECTION_NAME = (
    "IBS"  # @param {type:"string", "placeholder": "your-weaviate-collection-name"}
)
API_KEY = (
    "projects/ragdemo-453517/secrets/weaviate-api-key/versions/1"  # @param {type:"string", "placeholder": "your-secret-manager-resource-name"}
)
vector_db = rag.Weaviate(
    weaviate_http_endpoint=WEAVIATE_HTTP_ENDPOINT,
    collection_name=COLLECTION_NAME,
    api_key=API_KEY,
)

rag_corpus = rag.create_corpus(
    display_name=DISPLAY_NAME,
    embedding_model_config=embedding_model_config,
    vector_db=vector_db,
)

In [6]:
# Check the corpus just created
new_corpus = rag.get_corpus(name=rag_corpus.name)
new_corpus

RagCorpus(name='projects/895528006958/locations/us-central1/ragCorpora/3458764513820540928', display_name='IBS', description='', embedding_model_config=EmbeddingModelConfig(publisher_model='projects/ragdemo-453517/locations/us-central1/publishers/google/models/text-embedding-004', endpoint=None, model=None, model_version_id=None), vector_db=Weaviate(weaviate_http_endpoint='https://qdydcws8aopshqoyausw.c0.europe-west3.gcp.weaviate.cloud', collection_name='IBS', api_key='projects/ragdemo-453517/secrets/weaviate-api-key/versions/1'), vertex_ai_search_config=None, backend_config=RagVectorDbConfig(vector_db=None, rag_embedding_model_config=RagEmbeddingModelConfig(vertex_prediction_endpoint=VertexPredictionEndpoint(endpoint=None, publisher_model='projects/ragdemo-453517/locations/us-central1/publishers/google/models/text-embedding-004', model=None, model_version_id=None))))

## Upload a file to the corpus

In [7]:
%%writefile test.txt

Here's a demo for Weaviate RAG.

Overwriting test.txt


In [8]:
!gcloud secrets add-iam-policy-binding projects/ragdemo-453517/secrets/weaviate-api-key --member="serviceAccount:service-895528006958@gcp-sa-vertex-rag.iam.gserviceaccount.com"  --role="roles/secretmanager.secretAccessor"

Updated IAM policy for secret [weaviate-api-key].
bindings:
- members:
  - serviceAccount:service-895528006958@gcp-sa-vertex-rag.iam.gserviceaccount.com
  role: roles/secretmanager.secretAccessor
etag: BwYwKw_sc0I=
version: 1


In [9]:
rag_file = rag.upload_file(
    corpus_name=rag_corpus.name,
    path="test.txt",
    display_name="test.txt",
    description="my test",
)

## Import files from Google Cloud Storage

Remember to grant "Viewer" access to the "Vertex RAG Data Service Agent" (with the format of `service-{project_number}@gcp-sa-vertex-rag.iam.gserviceaccount.com`) for your Google Cloud Storage bucket

In [18]:
GCS_BUCKET = "https://drive.google.com/drive/folders/1kI_1LAEqooY6bshc6rnxhB7-9JvT5mJq"  # @param {type:"string", "placeholder": "your-gs-bucket"}

response = rag.import_files(  # noqa: F704
    corpus_name=rag_corpus.name,
    paths=[GCS_BUCKET],
    chunk_size=512,
    chunk_overlap=50,
)

In [19]:
# Check the files just imported. It may take a few seconds to process the imported files.
rag.list_files(corpus_name=rag_corpus.name)

ListRagFilesPager<rag_files {
  direct_upload_source {
  }
  name: "projects/895528006958/locations/us-central1/ragCorpora/3458764513820540928/ragFiles/5388000931346573235"
  display_name: "test.txt"
  description: "my test"
  create_time {
    seconds: 1741811390
    nanos: 951991000
  }
  update_time {
    seconds: 1741811391
    nanos: 127252000
  }
  file_status {
    state: ACTIVE
  }
}
>

## Import files from Google Drive

Eligible paths can be:

- `https://drive.google.com/drive/folders/{folder_id}`
- `https://drive.google.com/file/d/{file_id}`

Remember to grant "Viewer" access to the "Vertex RAG Data Service Agent" (with the format of `service-{project_number}@gcp-sa-vertex-rag.iam.gserviceaccount.com`) for your Drive folder/files.


In [20]:
FILE_ID = "1W4fWX9C-5MjS0X6lI1iP13BQ_TdMUGHC"# @param {type:"string", "placeholder": "your-file-id"}
FILE_PATH = f"https://drive.google.com/drive/folders/{FILE_ID}"

rag.import_files(
    corpus_name=rag_corpus.name,
    paths=[FILE_PATH],
    chunk_size=1024,
    chunk_overlap=100,
)

In [21]:
# Check the files just imported. It may take a few seconds to process the imported files.
rag.list_files(corpus_name=rag_corpus.name)

ListRagFilesPager<rag_files {
  direct_upload_source {
  }
  name: "projects/895528006958/locations/us-central1/ragCorpora/3458764513820540928/ragFiles/5388000931346573235"
  display_name: "test.txt"
  description: "my test"
  create_time {
    seconds: 1741811390
    nanos: 951991000
  }
  update_time {
    seconds: 1741811391
    nanos: 127252000
  }
  file_status {
    state: ACTIVE
  }
}
>

## Using Gemini GenerateContent API with Rag Retrieval Tool

When retrieved contexts similarity distance < `vector_distance_threshold`, the contexts (from `RagStore`) will be used for content generation.

In [15]:
rag_resource = rag.RagResource(
    rag_corpus=rag_corpus.name,
)

rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[rag_resource],  # Currently only 1 corpus is allowed.
            similarity_top_k=10,
            vector_distance_threshold=0.4,
        ),
    )
)

rag_model = GenerativeModel("gemini-1.5-flash", tools=[rag_retrieval_tool])

In [16]:
GENERATE_CONTENT_PROMPT = "What is RAG and why it is helpful?"  # @param {type:"string"}

response = rag_model.generate_content(GENERATE_CONTENT_PROMPT)

response

candidates {
  content {
    role: "model"
    parts {
      text: "I am sorry, but the provided sources do not contain information about what RAG is and why it is helpful. \n"
    }
  }
  avg_logprobs: -0.26023521025975543
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.078125
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0864257812
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.0275878906
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0100097656
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.0583496094
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0260009766
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.0756835938
    severity: HARM_SEVERITY

## Using other generation API with Rag Retrieval Tool

The retrieved contexts can be passed to any SDK or model generation API to generate final results.

In [17]:
RETRIEVAL_QUERY = "What is RAG and why it is helpful?"  # @param {type:"string"}

rag_resource = rag.RagResource(
    rag_corpus=rag_corpus.name,
    # Need to manually get the ids from rag.list_files.
    # rag_file_ids=[],
)

response = rag.retrieval_query(
    rag_resources=[rag_resource],  # Currently only 1 corpus is allowed.
    text=RETRIEVAL_QUERY,
    similarity_top_k=10,
)

# The retrieved context can be passed to any SDK or model generation API to generate final results.
retrieved_context = " ".join(
    [context.text for context in response.contexts.contexts]
).replace("\n", "")

retrieved_context

"Here's a demo for Weaviate RAG. Here's a demo for Weaviate RAG."

## Cleaning up

Clean up resources created in this notebook.

In [ ]:
delete_rag_corpus = False  # @param {type:"boolean"}

if delete_rag_corpus:
    rag.delete_corpus(name=rag_corpus.name)